<a href="https://colab.research.google.com/github/alenzhasulan/dl-nlp/blob/master/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import json
import numpy as np
import re
import collections

In [0]:
#Загружаем данные
JSONFile = open('data.json')
data = json.loads(JSONFile.read())
JSONFile.close()


train_source=data

In [0]:
TOKEN_RE=re.compile(r'[\w\d]+')

#Возвращает для каждого строку 
def tokenize_text_regex(txt,min_token_size=3):
  txt=txt.lower()
  all_tokens=TOKEN_RE.findall(txt)
  return [token for token in all_tokens if len(token)>=min_token_size]

#Для токенезаций
def tokenize_corpus(texts,tokenizer=tokenize_text_regex,**tokenizer_kwargs):
  return [tokenizer(text,**tokenizer_kwargs) for text in texts]

In [0]:
tokinezed_corpus=tokenize_corpus(train_source)

In [0]:
def build_vocabulary(tokenized_texts,max_size=10000000,max_doc_freq=0.8,min_count=5,pad_word=None):
  word_counts=collections.defaultdict(int)
  doc_n=0

  # подсчитать количество документов, в которых употребляется слова каждое слово
  # а также общее количество документов
  for txt in tokenized_texts:
    doc_n+=1
    unique_text_tokens=set(txt)
    for token in unique_text_tokens:
      word_counts[token]+=1
  
  # убрать слишком редкие и слишком частые слова
  word_counts={word:cnt for word,cnt in word_counts.items() if cnt>=min_count and cnt/doc_n<=max_doc_freq}

  # отсортировать слова по убыванию частоте 
  sorted_word_counts=sorted(word_counts.items(),reverse=True,key=lambda pair:pair[1])

  #добавим несуществующие слова с индексом 0 для удобство пакетной отроботки
  if pad_word is not None:
    sorted_word_counts=[(pad_word,0)]+sorted_word_counts

  #если у нас много слов от оставить самых частотных
  if len(word_counts)>max_size:
    sorted_word_counts=sorted_word_counts[:max_size]

  #нумируем слова
  word2id={word:i for i ,(word,_) in enumerate(sorted_word_counts)}

  #нормируем частотных слов
  word2freq=np.array([cnt/doc_n for _,cnt in sorted_word_counts],dtype='float32')

  return word2id,word2freq
  


In [0]:
MAX_DF=0.8
MIN_COUNT=5

vocabulary, word_doc_freq=build_vocabulary(tokinezed_corpus,
                                           max_doc_freq=MAX_DF,
                                           min_count=5
                                           )
UNIQUE_WORDS_N=len(vocabulary)
# print('Количество уникальных токенов ',UNIQUE_WORDS_N)
# print(list(vocabulary.items())[:10])